In [ ]:
'''
Author: yaobinsu 68414437+yaobinsu@users.noreply.github.com
Date: 2023-06-13 17:29:48
LastEditors: yaobinsu 68414437+yaobinsu@users.noreply.github.com
LastEditTime: 2023-06-13 17:29:59
FilePath: /wrs/wrs.ipynb
Description: 这是默认设置,请设置`customMade`, 打开koroFileHeader查看配置 进行设置: https://github.com/OBKoro1/koro1FileHeader/wiki/%E9%85%8D%E7%BD%AE
'''


## Exercise 1
